# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


Timelink version: 1.1.8
Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: dehergne_locations
Kleio image: timelinkserver/kleio-server
Kleio server token: eckih...
Kleio server URL: http://127.0.0.1:8088
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: vigilant_bartik
Kleio version requested: latest
Kleio server version: 12.4.567 (2024-02-07 13:02:03)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne-locations/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=dehergne_locations, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [2]:
tlnb.table_row_count_df()

,table,count
0,acts,40
1,attributes,15297
2,class_attributes,63
3,classes,13
4,entities,18644
5,geoentities,359
6,goods,0
7,kleiofiles,5
8,objects,0
9,persons,1066


### Show the kleio files available 

In [2]:
kleio_files = tlnb.get_kleio_files()
kleio_files

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/dehergne-a.cli,dehergne-a.cli,2024-05-02 06:35:19.086758+00:00,V,2024-05-02 06:35:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-02 06:35:42.675409,/rest/reports/sources/dehergne-a.rpt,/rest/exports/sources/dehergne-a.xml
1,sources/dehergne-b.cli,dehergne-b.cli,2024-04-24 07:10:22.277959+00:00,V,2024-04-24 07:10:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-24 07:12:17.349028,/rest/reports/sources/dehergne-b.rpt,/rest/exports/sources/dehergne-b.xml
2,sources/dehergne-c.cli,dehergne-c.cli,2024-04-30 01:45:48.250870+00:00,V,2024-04-27 13:41:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-30 01:59:58.134706,/rest/reports/sources/dehergne-c.rpt,/rest/exports/sources/dehergne-c.xml
3,sources/dehergne-d.cli,dehergne-d.cli,2024-04-30 01:45:48.258946+00:00,V,2024-04-28 12:25:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-30 02:00:08.739021,/rest/reports/sources/dehergne-d.rpt,/rest/exports/sources/dehergne-d.xml
4,sources/dehergne-e.cli,dehergne-e.cli,2024-04-24 07:10:24.775165+00:00,V,2024-04-24 07:10:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-24 07:12:55.410734,/rest/reports/sources/dehergne-e.rpt,/rest/exports/sources/dehergne-e.xml
5,sources/dehergne-f.cli,dehergne-f.cli,2024-04-30 01:45:48.269296+00:00,V,2024-04-29 12:01:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-30 02:00:25.315864,/rest/reports/sources/dehergne-f.rpt,/rest/exports/sources/dehergne-f.xml
6,sources/dehergne-g.cli,dehergne-g.cli,2024-04-24 07:10:38.282136+00:00,V,2024-04-24 07:10:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-24 07:13:26.858504,/rest/reports/sources/dehergne-g.rpt,/rest/exports/sources/dehergne-g.xml
7,sources/dehergne-h.cli,dehergne-h.cli,2024-04-24 07:10:35.268574+00:00,V,2024-04-24 07:10:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-24 07:13:31.803220,/rest/reports/sources/dehergne-h.rpt,/rest/exports/sources/dehergne-h.xml
8,sources/dehergne-i.cli,dehergne-i.cli,2024-04-24 07:10:37.962737+00:00,V,2024-04-24 07:10:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-24 07:13:34.285669,/rest/reports/sources/dehergne-i.rpt,/rest/exports/sources/dehergne-i.xml
9,sources/dehergne-j.cli,dehergne-j.cli,2024-04-30 01:45:48.276581+00:00,V,2024-04-27 13:40:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-30 02:00:28.338193,/rest/reports/sources/dehergne-j.rpt,/rest/exports/sources/dehergne-j.xml


## Optionally clean translations 

In [9]:
tlnb.kleio_server.translation_clean("", recurse="yes")

['kleio/dehergne-locations-1701.cli',
 'kleio/dehergne-locations-1644.cli',
 'kleio/dehergne-a.cli',
 'kleio/b1685.cli',
 'kleio/auc-alunos.cli']


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [5]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()

INFO:root:Request translation of T sources/dehergne-r.cli
INFO:root:Importing sources/dehergne-r.cli
INFO:root:Importing sources/dehergne-r.cli
INFO:root:Storing 31 postponed relations


## Check the status of the files

Check the import status of the translated files

I: Imported

E: Imported with error

W: Imported with warnings no errors

N: Not imported

U: Translation updated need to reimport

In [6]:
imported_files_df = tlnb.get_import_status()
imported_files_df[["import_status","import_errors","import_warnings","name","errors","warnings","path"]].sort_values("name")


,import_status,import_errors,import_warnings,name,errors,warnings,path
0,I,0,0,dehergne-a.cli,0,0,sources/dehergne-a.cli
1,I,0,0,dehergne-b.cli,0,0,sources/dehergne-b.cli
2,I,0,0,dehergne-c.cli,0,0,sources/dehergne-c.cli
3,I,0,0,dehergne-d.cli,0,0,sources/dehergne-d.cli
4,I,0,0,dehergne-e.cli,0,0,sources/dehergne-e.cli
5,I,0,0,dehergne-f.cli,0,0,sources/dehergne-f.cli
6,I,0,0,dehergne-g.cli,0,0,sources/dehergne-g.cli
7,I,0,0,dehergne-h.cli,0,0,sources/dehergne-h.cli
8,I,0,0,dehergne-i.cli,0,0,sources/dehergne-i.cli
9,I,0,0,dehergne-j.cli,0,0,sources/dehergne-j.cli


### Check a translation report

If needed to check for errors.


In [7]:
file_number=2
rpt = tlnb.get_translation_report(imported_files_df, file_number)
print(rpt)


HTTPError: HTTP Error 400: Bad Request

### Get the import report for a file

In case there are errors in the import phase check the import report.

In [ ]:
rpt = tlnb.get_import_rpt(imported_files_df, rows=[0])
print(rpt)

No errors
No warnings
